In [1]:
import pandas as pd
import re
from datetime import datetime, timedelta
import os

# --- CONFIGURATION ---
FILE = 'CSV FILE NAME'
FILE1 = ' '
FILE2 = ' '
OUTPUT_FILE = 'OUTPUT CSV FILE NAME'

# Define Categories
driver_bins = {
    'Music & DJ': ['music', 'dj', 'sound', 'song', 'edm', 'hip hop', 'pop', 'techno', 'playlist', 'audio', 'speaker'],
    'Drinks & Alcohol': ['drink', 'alcohol', 'cocktail', 'beer', 'bar', 'unli', 'free flow', 'water', 'weak', 'fake', 'strong', 'wine', 'champagne', 'shot', 'drunk', 'sober', 'tipsy', 'bartender'],
    'Crowd & Atmosphere': ['crowd', 'packed', 'squeeze', 'people', 'vibe', 'atmosphere', 'smoke', 'hot', 'air', 'ventilation', 'space', 'dance floor', 'stuffy', 'busy', 'empty', 'dead', 'lively', 'fun', 'exciting'],
    'Service & Staff': ['service', 'staff', 'pr', 'bouncer', 'security', 'rude', 'friendly', 'polite', 'attitude', 'guard', 'waiter', 'manager', 'reception', 'door'],
    'Price & Entry': ['price', 'cost', 'entry', 'ticket', 'cover', 'expensive', 'cheap', 'worth', 'ntd', 'pay', 'money', 'scam', 'rip off', 'deal', 'free'],
    'Facilities & Safety': ['toilet', 'bathroom', 'restroom', 'clean', 'dirty', 'safe', 'unsafe', 'thief', 'stole', 'lost', 'lockers', 'ac', 'smoking']
}

def categorize_review(text):
    text = str(text).lower()
    found_drivers = []
    for driver, keywords in driver_bins.items():
        if any(word in text for word in keywords):
            found_drivers.append(driver)
    return found_drivers if found_drivers else ["Uncategorized"]

def parse_relative_date(row):
    rel_str = str(row['relative_date']).lower().strip()
    try:
        scrape_date = pd.to_datetime(row['scrape_date'])
    except:
        scrape_date = datetime.now()

    if rel_str in ['just now', 'unknown', 'nan', '']:
        return scrape_date

    if 'a ' in rel_str or 'an ' in rel_str: number = 1
    else:
        match = re.search(r'\d+', rel_str)
        number = int(match.group()) if match else 0

    if 'minute' in rel_str: delta = timedelta(minutes=number)
    elif 'hour' in rel_str: delta = timedelta(hours=number)
    elif 'day' in rel_str: delta = timedelta(days=number)
    elif 'week' in rel_str: delta = timedelta(weeks=number)
    elif 'month' in rel_str: delta = timedelta(days=number * 30) 
    elif 'year' in rel_str: delta = timedelta(days=number * 365)
    else: delta = timedelta(days=0)
    return scrape_date - delta

# --- EXECUTION ---
print("Loading files...")
dfs = []
try:
    if os.path.exists(FILE1):
        dfs.append(pd.read_csv(FILE1))

    if os.path.exists(FILE2):
        dfs.append(pd.read_csv(FILE2))

    if not dfs:
        print("Error: No files found to process.")
        exit()
except:
    pass

try:
    df = pd.read_csv(FILE)
    print(f"Total Raw Rows: {len(df)}")
except Exception as e:
    print(f'Error loading file: {e}')
    raise 

# Remove Duplicates 
df = df.drop_duplicates(subset=['review_text', 'relative_date', 'rating'])
print(f"Unique Reviews: {len(df)}")

if 'scrape_date' not in df.columns:
    df['scrape_date'] = datetime.now().strftime("%Y-%m-%d")

df['real_date'] = df.apply(parse_relative_date, axis=1)

# Clean Text
df = df.dropna(subset=['review_text'])
df = df[df['review_text'].str.strip() != '']

# Fix Ratings
df['rating'] = df['rating'].astype(str).str.replace(r'\s*stars?', '', regex=True)
df['rating'] = pd.to_numeric(df['rating'], errors='coerce').fillna(0).astype(int)

# Categorize
print("Categorizing reviews...")
df['drivers_list'] = df['review_text'].apply(categorize_review)

# Filter Useless Reviews
df['word_count'] = df['review_text'].apply(lambda x: len(str(x).split()))
df = df[ ~((df['drivers_list'].apply(lambda x: 'Uncategorized' in x)) & (df['word_count'] < 5)) ]

# Explode
print("Exploding dataset...")
df_exploded = df.explode('drivers_list').rename(columns={'drivers_list': 'category', 'real_date': 'review_date'})
df_exploded.drop(columns='relative_date', inplace=True)
df_exploded = df_exploded[df_exploded['category'] != 'Uncategorized']

# Save
df_exploded.to_csv(OUTPUT_FILE, index=False)
print(f"\nSUCCESS! Created '{OUTPUT_FILE}' with {len(df_exploded)} rows.")

Loading files...
Total Raw Rows: 431
Unique Reviews: 431
Categorizing reviews...
Exploding dataset...

SUCCESS! Created 'Frank_reviews_clean_exploded_final.csv' with 789 rows.
